# Load Training Data

In [1]:
import pathlib
import pandas as pd

# set your path here
data_root = pathlib.Path('/Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/')
manifest_path = data_root/'01_manifest.pkl'

# read the manifest in
manifest = pd.read_pickle(manifest_path)
manifest.head()

,filename,package_hash,manual_tag,tag_set,category_set
0,101729-0-0-1.wav,u8k_fold9,air_conditioner,{air_conditioner},{mechanical}
1,103249-5-0-1.wav,u8k_fold9,engine_idling,{engine_idling},{transport_car}
2,104817-4-0-11.wav,u8k_fold2,drilling,{drilling},{mechanical_construction}
3,104998-7-16-0.wav,u8k_fold5,jackhammer,{jackhammer},{mechanical_construction}
4,104998-7-18-3.wav,u8k_fold5,jackhammer,{jackhammer},{mechanical_construction}


# Category Labels

In [2]:
from sklearn.preprocessing import MultiLabelBinarizer

final_categories = ["animal_dogs", "animal_insects", "animal_birds", "animal_cockatoo", "animal_poultry",  "background","human_voice",  "indeterminate","mechanical", "mechanical_construction", "mechanical_impulsive","nature_wind", "signals_horn", "signals_siren", "transport_car"]
category_encoder = MultiLabelBinarizer().fit([final_categories])
category_encoder.classes_

array(['animal_birds', 'animal_cockatoo', 'animal_dogs', 'animal_insects',
       'animal_poultry', 'background', 'human_voice', 'indeterminate',
       'mechanical', 'mechanical_construction', 'mechanical_impulsive',
       'nature_wind', 'signals_horn', 'signals_siren', 'transport_car'],
      dtype=object)

# Preprocessing and Features

In [3]:
# These spectrogram settings look pretty good from a domain perspective.
# Trying a little bit higher "resolution" than previously
mel_settings = {'fmax': 8000, 'power': 2, 'n_mels' :128, 'n_fft':2048, 'hop_length':512}

In [4]:
import os
import numpy as np
import soundfile as sf
import librosa
import librosa.display
import sklearn
def get_mels(filepath='', data=[], fs=-1):
    if filepath:
        data, fs = sf.read(filepath)
    else:
        assert (len(data>0) and fs >0), 'Must provide either a filename, or array of data and sample rate'
    
    S = librosa.feature.melspectrogram(y=data, 
                                   sr = fs, **mel_settings)
    return S, fs
 
def load_mels(filepath, force_create=False, save=True):
    mel_path = filepath.with_suffix('.npy')
    
    if mel_path.is_file() and not force_create:
        print('Loading {}'.format(mel_path))
        mels = np.load(mel_path)
    else:
        print('Generating from {}'.format(filepath))
        mels, _ = get_mels(filepath)
        if save:
            print('Saving {}'.format(mel_path))
            np.save(mel_path, mels)
    
    return mels

def feature_preprocessing(mel):
    # convert to db and normalise
    power = librosa.core.power_to_db(mel, ref=np.max)
    power = power - np.mean(power)
    power = power / (np.std(power))
    return power[:, :, None]


In [5]:
# generate the features
# note this will store all features in memory, as well as saving them to disk. 
# Can't guarantee it will work for large datasets.
manifest['features'] = manifest.apply(lambda x: data_root/x['package_hash']/x['filename'], axis=1).apply(lambda x: feature_preprocessing(load_mels(x, force_create=True, save=True)))

Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold9/101729-0-0-1.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold9/101729-0-0-1.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold9/103249-5-0-1.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold9/103249-5-0-1.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold2/104817-4-0-11.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold2/104817-4-0-11.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold5/104998-7-16-0.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_

Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold3/18594-1-5-0.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold7/186936-5-0-0.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold7/186936-5-0-0.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold4/187356-1-4-0.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold4/187356-1-4-0.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold2/189023-0-0-3.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold2/189023-0-0-3.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/u8k_fold

Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/637_20190115_cnnmodel_verification_package/2018-12-24 22_04_19.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/637_20190115_cnnmodel_verification_package/2018-12-24 22_52_58.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/637_20190115_cnnmodel_verification_package/2018-12-24 22_52_58.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/637_20190115_cnnmodel_verification_package/2018-12-29 20_24_14.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/637_20190115_cnnmodel_verification_package/2018-12-29 20_24_14.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/637_20190115_cnnmodel_verification_package/2018-12-29 20_43_59

Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/648_cnnmodel_20190221_verification_package/2019-02-09 08_03_21.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/648_cnnmodel_20190221_verification_package/2019-02-09 08_10_21.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/648_cnnmodel_20190221_verification_package/2019-02-09 08_10_21.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/648_cnnmodel_20190221_verification_package/2019-02-09 08_52_53.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/648_cnnmodel_20190221_verification_package/2019-02-09 08_52_53.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/648_cnnmodel_20190221_verification_package/2019-02-09 20_36_38

Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/VdXN/2019-02-19 10_22_41.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/VdXN/2019-02-19 10_34_03.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/VdXN/2019-02-19 10_34_03.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/635_cnnmodel_20190220_verification_package/2019-02-19 19_28_50.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/635_cnnmodel_20190220_verification_package/2019-02-19 19_28_50.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/635_cnnmodel_20190220_verification_package_subset/2019-02-19 19_28_54.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/635_cn

...

Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ugtf/2022-01-28T11_15_09+1000.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ugtf/2022-01-28T20_35_14+1000.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ugtf/2022-01-28T20_35_14+1000.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/iept/2022-01-29T08_01_21+1000.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/iept/2022-01-29T08_01_21+1000.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ugtf/2022-01-30T03_14_18+1000.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ugtf/2022-01-30T03_14_18+1000.npy
Generating from /Volumes/Clavius/documents/Documents/Employment

Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/zgvm/2022-02-06T09_38_15.115374+09_30.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ohff/2022-02-06T17_36_33.917722+09_30.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ohff/2022-02-06T17_36_33.917722+09_30.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-06T17_52_28+0930.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-06T17_52_28+0930.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ohff/2022-02-06T17_55_17.634888+09_30.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/ohff/2022-02-06T17_55_17.634888+09_30.npy
Generating from /Volume

Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/eviz/2022-02-12T00_07_16+0930.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/eviz/2022-02-12T00_07_16+0930.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-12T07_26_26+0930.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-12T07_26_26+0930.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/tkgr/2022-02-12T10_15_43.225147+09_30.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/tkgr/2022-02-12T10_15_43.225147+09_30.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-12T16_20_26+0930.wav
Saving /Volumes/Clavius/documents/Docu

Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-22T08_36_12+0930.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-22T08_36_12+0930.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/eviz/2022-02-23T16_16_24+0930.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/eviz/2022-02-23T16_16_24+0930.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-24T01_01_03+0930.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/uift/2022-02-24T01_01_03+0930.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/jnam/2022-03-02T15_22_11+1000.wav
Saving /Volumes/Clavius/documents/Documents/Employment

Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/bgcs/2022-03-14T00_52_10+0800.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/bgcs/2022-03-14T01_03_34+0800.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/bgcs/2022-03-14T01_03_34+0800.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/bgcs/2022-03-14T08_24_38+0800.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/bgcs/2022-03-14T08_24_38+0800.npy
Generating from /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/bgcs/2022-03-14T19_19_33+0800.wav
Saving /Volumes/Clavius/documents/Documents/Employment/NoiseNet/Development/tag_data/sample_set/bgcs/2022-03-14T19_19_33+0800.npy
Generating from /Volumes/Clavius/documents/Documents/Employment

In [6]:
manifest.iloc[0]['features'].shape

(128, 126, 1)

# Test/Train Split

NB
* Category Supports reflect rough proportions in the large data set. 
* Will have to improve design by balancing classes via oversampling, augmentation, or training weights.

In [7]:
from sklearn.model_selection import train_test_split

X = manifest['features'].values
y = category_encoder.transform(manifest['category_set'].values)

print('Category Support')
for c,n in zip(category_encoder.classes_, y.sum(axis=0)):
    print('{:30s}{} : {}'.format(c, category_encoder.transform([[c]]), n) )

print()
print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Category Support
animal_birds                  [[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] : 2382
animal_cockatoo               [[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]] : 6
animal_dogs                   [[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]] : 2229
animal_insects                [[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]] : 677
animal_poultry                [[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]] : 12
background                    [[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]] : 915
human_voice                   [[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]] : 964
indeterminate                 [[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]] : 17
mechanical                    [[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]] : 2099
mechanical_construction       [[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]] : 35
mechanical_impulsive          [[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]] : 4
nature_wind                   [[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]] : 856
signals_horn                  [[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]] : 7
signals_siren                 [[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]] : 7
transport_c

/opt/anaconda3/envs/ldi/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:875: UserWarning: unknown class(es) ['animal_other', 'human_movement', 'music', 'signals_bell', 'transport_aircraft', 'transport_motorcycle'] will be ignored
  warnings.warn(
